# RLS Consumer 

## Goal


The goal of this notebook is to perform Task 3 and 4 : that is to apply the RLS and the Time augmented RLS on the stream produced by the Producer.

This processing will be done separetaly for each models :
- **RLS N=1**
- **RLS N=2**
- **RLS N=3**
- **RLS N=4**
- **Time related RLS N=4** : this one is done because RLS N=4 was shown to be the best performing vanilla RLS

Also for a same model, say RLS N=2, we will maintain 1 state/model per Sensor. This will be enabled by the *updateStateByKey()* function.
Each state will contain :
1. the key : the name of the Sensor
2. the previous values in an array
3. the number of predictions
4. the sum of the squarred error (SSE)
5. the number of time we have the same prediction error
6. RLS parameters : a dictionary containing all the parameters of the RLS
    1. beta
    2. V
    3. nu

Performance Results can be seen in **Model_Racing_Plots.ipynb**


## Common imports

In [1]:
import pandas as pd
import numpy as np
import ast
from pyspark import Row
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from itertools import combinations
from pyspark.sql.types import StructField, IntegerType, FloatType
import os
import math 
from pyspark.ml.evaluation import RegressionEvaluator

os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 ' + \
                                    '--conf spark.driver.memory=1g  pyspark-shell '
streaming_path = "streaming"

## Spark session construction

In [2]:
#This function creates a connection to a network socket
#You may change the batch interval
#The function returns the Spark context, Spark streaming context, and DStream object
SOCKET = 2222
def getDStream(spark, batch_interval=10):

    # Get Spark context
    sc = spark.sparkContext
    sc.setLogLevel("ERROR")

    #Create streaming context, with required batch interval
    ssc = StreamingContext(sc, batch_interval)

    #Checkpointing needed for stateful transforms
    ssc.checkpoint("checkpoint")
    
    # Create a DStream that represents streaming data from a network socket
    # See https://spark.apache.org/docs/latest/streaming-programming-guide.html#a-quick-example
    dstream = ssc.socketTextStream("localhost", SOCKET)
    
    return [sc,ssc,dstream]

In [3]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("bike_forecasting") \
    .getOrCreate()
sc = spark.sparkContext

23/05/14 11:28:05 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.129.3 instead (on interface enp38s0)
23/05/14 11:28:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/14 11:28:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/14 11:28:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Stateful Processing of RLS models

First we retrieve the different sensors to define the different keys of the states

In [4]:
df = spark.read.option("header", True).csv("data.csv")
different_sensors = df.filter("Date =='2022-01-01'").filter(col("Time Gap") == 1).select(col("Sensor")).distinct().toPandas().values


## RLS Computation Vanilla
Defines the RLS vanilla (without time related features)

In [29]:
def set_RLS_Vanilla_state_from_x(x, previous_value_size):
    """
    Define the state value (key not included) from x (the row received from producer).
    @param x : the line received from the producer.
    @param previous_value_size : the number of previous value we want to keep track of. (Equal to the n parameter in the statement)
    """
    previous_values = np.zeros(previous_value_size)
    nb_predictions = 0
    SSE = 0
    last_prediction = 0
    nb_same_SSE = 0
    v0 = 1
    RLS_parameters = {
        "beta" : np.zeros(previous_value_size + 1),
        "V" :  np.diag(np.zeros(previous_value_size + 1) + v0),
        "nu" : 1
    }
    
    return (x[0],last_prediction, previous_values,nb_predictions,SSE, nb_same_SSE, RLS_parameters)

def RLSstepVanilla(y, x, n, beta, V, nu):
    """
    Perform one step of the RLS
    @param y : the true value (to compute the error)
    @param x : the previous values
    @param n : the size of previous values
    @param beta : beta
    @param V: V
    @param nu: nu
    """
    x = x.copy()
    x.shape = (1, n)
    x1 = np.append(1, x)
    x1.shape = (1, n+1)
    
    V_up = V.dot(x1.T).dot(x1).dot(V)
    V_down = (1.0+float(x1.dot(V).dot(x1.T)))
    
    V = (1.0/nu) * (V - V_up/V_down)
    alpha = V.dot(x1.T)

    yhat = x1.dot(beta)

    err = y - yhat[0]
    
    beta = beta + alpha * err

    return (beta, V, err, yhat[0])


def updateFunctionRLS(newValues, state):
    """   
    Update the state of the corresponding sensor with newValues.
    Perform the prediction and updtate the metrics
    @param newValues : the new values from the stream : one row of the corresponding sensor
    @param state : the current state to be updated
    """
    nb_values = len(newValues)
    if nb_values > 0:
        for line in newValues:            
            key, last_prediction, previous_values, nb_predictions, SSE, nb_same_SEE, RLS_parameters = state 
            true_value = line[1][4]
            n = len(previous_values)
            RLS_parameters["beta"].shape=(n+1,1)
            RLS_parameters["beta"], RLS_parameters["V"], err, prediction = RLSstepVanilla(float(true_value), previous_values, n, RLS_parameters["beta"],RLS_parameters["V"],RLS_parameters["nu"]) # to change for each model
    
            SSE += math.pow(err, 2)
            nb_predictions +=1
            
            previous_values = np.concatenate([previous_values[1:],[float(true_value)]])
            date = line[1][1]
           
            state = (key, prediction, previous_values, nb_predictions, SSE,0, RLS_parameters)
    else:
        #this block is used to set the state to none if we have 5 time the same SEE (we do not receive new data 5 times in a row)
        key, last_prediction, previous_values, nb_predictions, SSE, nb_same_SEE, RLS_parameters = state 
        nb_same_SEE +=1
        if nb_same_SEE >= 5:
            return None
        state = (key, last_prediction, previous_values, nb_predictions, SSE, nb_same_SEE, RLS_parameters)
    return state


### RLS N=1

In [38]:
N = 1

batch_interval = 2
[sc,ssc,dstream] = getDStream(spark, batch_interval)
initialStateRDD = sc.parallelize([(x[0], set_RLS_Vanilla_state_from_x(x, N)) for x in different_sensors]) 
dataS=dstream.map(ast.literal_eval)


dataS=dataS.flatMap(lambda x: [(x[3],(x[3],x))]) #x[3] contains the key (sensor)
dataS.pprint()
updatedState = dataS.updateStateByKey(updateFunctionRLS, initialRDD=initialStateRDD)

def save_to_file(rdd):
    """
    Update the state on a file to retrieve data after the stream completed
    """
    # coalesce the RDD into a single partition
    merged_rdd = rdd.coalesce(1)
    # save the merged RDD to a text file
    if not merged_rdd.isEmpty():
        merged_rdd.map(lambda x: f"{x[0]},{(x[1][4]/(x[1][3] if x[1][3] != 0 else 1)):.2f},{x[1][4]},{x[1][3]}") \
            .saveAsTextFile("streaming/RLS_Vanilla_N_1")


updatedState.foreachRDD(lambda x : save_to_file(x))
ssc.start()


-------------------------------------------
Time: 2023-05-14 10:04:56
-------------------------------------------
('CB1101', ('CB1101', ['2120258', '2022-04-16', '1', 'CB1101', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1142', ('CB1142', ['2120259', '2022-04-16', '1', 'CB1142', '3.0', '21.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1143', ('CB1143', ['2120260', '2022-04-16', '1', 'CB1143', '1.0', '27.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1599', ('CB1599', ['2120261', '2022-04-16', '1', 'CB1599', '1.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1699', ('CB1699', ['2120262', '2022-04-16', '1', 'CB1699', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB2105', ('CB2105', ['2120263', '2022-04-16', '1', 'CB2105', '7.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CEE016', ('CEE016', ['2120264', '2022-04-16', '1', 'CEE016', '1.0', '19.0'

-------------------------------------------
Time: 2023-05-14 10:04:58
-------------------------------------------
('CAT17', ('CAT17', ['2123712', '2022-04-18', '1', 'CAT17', '2.0', '17.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB02411', ('CB02411', ['2123713', '2022-04-18', '1', 'CB02411', '2.0', '28.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1101', ('CB1101', ['2123714', '2022-04-18', '1', 'CB1101', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1142', ('CB1142', ['2123715', '2022-04-18', '1', 'CB1142', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1143', ('CB1143', ['2123716', '2022-04-18', '1', 'CB1143', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1599', ('CB1599', ['2123717', '2022-04-18', '1', 'CB1599', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1699', ('CB1699', ['2123718', '2022-04-18', '1', 'CB1699', '1.0', '17.0'

-------------------------------------------
Time: 2023-05-14 10:05:00
-------------------------------------------
('CEK049', ('CEK049', ['2132091', '2022-04-22', '82', 'CEK049', '20.0', '19.0', '112', '6', '2022', '16', '2022-112', '16', '2022-16']))
('CEK18', ('CEK18', ['2132092', '2022-04-22', '82', 'CEK18', '1.0', '5.0', '112', '6', '2022', '16', '2022-112', '16', '2022-16']))
('CEK31', ('CEK31', ['2132093', '2022-04-22', '82', 'CEK31', '11.0', '18.0', '112', '6', '2022', '16', '2022-112', '16', '2022-16']))
('CEV011', ('CEV011', ['2132094', '2022-04-22', '82', 'CEV011', '1.0', '12.0', '112', '6', '2022', '16', '2022-112', '16', '2022-16']))
('CJE181', ('CJE181', ['2132095', '2022-04-22', '82', 'CJE181', '3.0', '24.0', '112', '6', '2022', '16', '2022-112', '16', '2022-16']))
('CJM90', ('CJM90', ['2132096', '2022-04-22', '82', 'CJM90', '55.0', '17.0', '112', '6', '2022', '16', '2022-112', '16', '2022-16']))
('CLW239', ('CLW239', ['2132097', '2022-04-22', '82', 'CLW239', '7.0', '21.0'

-------------------------------------------
Time: 2023-05-14 10:05:02
-------------------------------------------
('CAT17', ('CAT17', ['2135808', '2022-04-25', '1', 'CAT17', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB02411', ('CB02411', ['2135809', '2022-04-25', '1', 'CB02411', '3.0', '22.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1101', ('CB1101', ['2135810', '2022-04-25', '1', 'CB1101', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1142', ('CB1142', ['2135811', '2022-04-25', '1', 'CB1142', '2.0', '17.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1143', ('CB1143', ['2135812', '2022-04-25', '1', 'CB1143', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1599', ('CB1599', ['2135813', '2022-04-25', '1', 'CB1599', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1699', ('CB1699', ['2135814', '2022-04-25', '1', 'CB1699', '1.0', '20.0'

-------------------------------------------
Time: 2023-05-14 10:05:04
-------------------------------------------
('CAT17', ('CAT17', ['2147904', '2022-05-02', '1', 'CAT17', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB02411', ('CB02411', ['2147905', '2022-05-02', '1', 'CB02411', '49.0', '17.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1101', ('CB1101', ['2147906', '2022-05-02', '1', 'CB1101', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1142', ('CB1142', ['2147907', '2022-05-02', '1', 'CB1142', '5.0', '20.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1143', ('CB1143', ['2147908', '2022-05-02', '1', 'CB1143', '5.0', '23.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1599', ('CB1599', ['2147909', '2022-05-02', '1', 'CB1599', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1699', ('CB1699', ['2147910', '2022-05-02', '1', 'CB1699', '0.0', '-1.0

...

-------------------------------------------
Time: 2023-05-14 10:07:22
-------------------------------------------
('CAT17', ('CAT17', ['2569536', '2023-01-01', '1', 'CAT17', '1.0', '9.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB02411', ('CB02411', ['2569537', '2023-01-01', '1', 'CB02411', '8.0', '17.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1101', ('CB1101', ['2569538', '2023-01-01', '1', 'CB1101', '17.0', '15.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1142', ('CB1142', ['2569539', '2023-01-01', '1', 'CB1142', '12.0', '18.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1143', ('CB1143', ['2569540', '2023-01-01', '1', 'CB1143', '16.0', '18.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1599', ('CB1599', ['2569541', '2023-01-01', '1', 'CB1599', '3.0', '19.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1699', ('CB1699', ['2569542', '2023-01-01', '1', 'CB1699', '2.0', '16.0', '1', '1', '2023', '5

-------------------------------------------
Time: 2023-05-14 10:07:24
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 10:07:26
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 10:07:28
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 10:07:30
-------------------------------------------



In [39]:
ssc.stop(stopSparkContext=False, stopGraceFully=False)

23/05/14 10:08:42 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.ja

-------------------------------------------
Time: 2023-05-14 10:07:32
-------------------------------------------



### RLS N=2

In [32]:
N = 2

batch_interval = 2
[sc,ssc,dstream] = getDStream(spark, batch_interval)
initialStateRDD = sc.parallelize([(x[0], set_RLS_Vanilla_state_from_x(x, N)) for x in different_sensors]) 
dataS=dstream.map(ast.literal_eval)


dataS=dataS.flatMap(lambda x: [(x[3],(x[3],x))]) #x[3] contains the key (sensor)
dataS.pprint()
updatedState = dataS.updateStateByKey(updateFunctionRLS, initialRDD=initialStateRDD)

def save_to_file(rdd):
    """
    Update the state on a file to retrieve data after the stream completed
    """
    # coalesce the RDD into a single partition
    merged_rdd = rdd.coalesce(1)
    # save the merged RDD to a text file
    if not merged_rdd.isEmpty():
        merged_rdd.map(lambda x: f"{x[0]},{(x[1][4]/(x[1][3] if x[1][3] != 0 else 1)):.2f},{x[1][4]},{x[1][3]}") \
            .saveAsTextFile("streaming/RLS_Vanilla_N_2")


updatedState.foreachRDD(lambda x : save_to_file(x))
ssc.start()

-------------------------------------------
Time: 2023-05-14 09:52:16
-------------------------------------------
('CB02411', ('CB02411', ['2120257', '2022-04-16', '1', 'CB02411', '3.0', '22.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1101', ('CB1101', ['2120258', '2022-04-16', '1', 'CB1101', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1142', ('CB1142', ['2120259', '2022-04-16', '1', 'CB1142', '3.0', '21.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1143', ('CB1143', ['2120260', '2022-04-16', '1', 'CB1143', '1.0', '27.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1599', ('CB1599', ['2120261', '2022-04-16', '1', 'CB1599', '1.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1699', ('CB1699', ['2120262', '2022-04-16', '1', 'CB1699', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB2105', ('CB2105', ['2120263', '2022-04-16', '1', 'CB2105', '7.0', '19

-------------------------------------------
Time: 2023-05-14 09:52:18
-------------------------------------------
('CAT17', ('CAT17', ['2123712', '2022-04-18', '1', 'CAT17', '2.0', '17.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB02411', ('CB02411', ['2123713', '2022-04-18', '1', 'CB02411', '2.0', '28.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1101', ('CB1101', ['2123714', '2022-04-18', '1', 'CB1101', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1142', ('CB1142', ['2123715', '2022-04-18', '1', 'CB1142', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1143', ('CB1143', ['2123716', '2022-04-18', '1', 'CB1143', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1599', ('CB1599', ['2123717', '2022-04-18', '1', 'CB1599', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1699', ('CB1699', ['2123718', '2022-04-18', '1', 'CB1699', '1.0', '17.0'

-------------------------------------------
Time: 2023-05-14 09:52:20
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 09:52:22
-------------------------------------------
('CAT17', ('CAT17', ['2135808', '2022-04-25', '1', 'CAT17', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB02411', ('CB02411', ['2135809', '2022-04-25', '1', 'CB02411', '3.0', '22.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1101', ('CB1101', ['2135810', '2022-04-25', '1', 'CB1101', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1142', ('CB1142', ['2135811', '2022-04-25', '1', 'CB1142', '2.0', '17.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1143', ('CB1143', ['2135812', '2022-04-25', '1', 'CB1143', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1599', ('CB1599', ['2135813', '2022-04-25', '1', 'CB1599', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1699', ('CB1699', ['2135814', '2022-04-25', '1', 'CB1699', '1.0', '20.0'

-------------------------------------------
Time: 2023-05-14 09:52:24
-------------------------------------------
('CAT17', ('CAT17', ['2147904', '2022-05-02', '1', 'CAT17', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB02411', ('CB02411', ['2147905', '2022-05-02', '1', 'CB02411', '49.0', '17.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1101', ('CB1101', ['2147906', '2022-05-02', '1', 'CB1101', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1142', ('CB1142', ['2147907', '2022-05-02', '1', 'CB1142', '5.0', '20.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1143', ('CB1143', ['2147908', '2022-05-02', '1', 'CB1143', '5.0', '23.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1599', ('CB1599', ['2147909', '2022-05-02', '1', 'CB1599', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1699', ('CB1699', ['2147910', '2022-05-02', '1', 'CB1699', '0.0', '-1.0

...

-------------------------------------------
Time: 2023-05-14 09:54:42
-------------------------------------------
('CAT17', ('CAT17', ['2569536', '2023-01-01', '1', 'CAT17', '1.0', '9.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB02411', ('CB02411', ['2569537', '2023-01-01', '1', 'CB02411', '8.0', '17.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1101', ('CB1101', ['2569538', '2023-01-01', '1', 'CB1101', '17.0', '15.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1142', ('CB1142', ['2569539', '2023-01-01', '1', 'CB1142', '12.0', '18.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1143', ('CB1143', ['2569540', '2023-01-01', '1', 'CB1143', '16.0', '18.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1599', ('CB1599', ['2569541', '2023-01-01', '1', 'CB1599', '3.0', '19.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1699', ('CB1699', ['2569542', '2023-01-01', '1', 'CB1699', '2.0', '16.0', '1', '1', '2023', '5

-------------------------------------------
Time: 2023-05-14 09:54:44
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 09:54:46
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 09:54:48
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 09:54:50
-------------------------------------------



In [33]:
ssc.stop(stopSparkContext=False, stopGraceFully=False)

23/05/14 09:56:06 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.ja

-------------------------------------------
Time: 2023-05-14 09:54:52
-------------------------------------------



### RLS N=3

In [34]:
N = 3

batch_interval = 2
[sc,ssc,dstream] = getDStream(spark, batch_interval)
initialStateRDD = sc.parallelize([(x[0], set_RLS_Vanilla_state_from_x(x, N)) for x in different_sensors]) 
dataS=dstream.map(ast.literal_eval)


dataS=dataS.flatMap(lambda x: [(x[3],(x[3],x))]) #x[3] contains the key (sensor)
dataS.pprint()
updatedState = dataS.updateStateByKey(updateFunctionRLS, initialRDD=initialStateRDD)

def save_to_file(rdd):
    """
    Update the state on a file to retrieve data after the stream completed
    """
    # coalesce the RDD into a single partition
    merged_rdd = rdd.coalesce(1)
    # save the merged RDD to a text file
    if not merged_rdd.isEmpty():
        merged_rdd.map(lambda x: f"{x[0]},{(x[1][4]/(x[1][3] if x[1][3] != 0 else 1)):.2f},{x[1][4]},{x[1][3]}") \
            .saveAsTextFile("streaming/RLS_Vanilla_N_3")


updatedState.foreachRDD(lambda x : save_to_file(x))
ssc.start()

-------------------------------------------
Time: 2023-05-14 09:56:18
-------------------------------------------
('CB1101', ('CB1101', ['2120258', '2022-04-16', '1', 'CB1101', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1142', ('CB1142', ['2120259', '2022-04-16', '1', 'CB1142', '3.0', '21.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1143', ('CB1143', ['2120260', '2022-04-16', '1', 'CB1143', '1.0', '27.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1599', ('CB1599', ['2120261', '2022-04-16', '1', 'CB1599', '1.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1699', ('CB1699', ['2120262', '2022-04-16', '1', 'CB1699', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB2105', ('CB2105', ['2120263', '2022-04-16', '1', 'CB2105', '7.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CEE016', ('CEE016', ['2120264', '2022-04-16', '1', 'CEE016', '1.0', '19.0'

-------------------------------------------
Time: 2023-05-14 09:56:20
-------------------------------------------
('CAT17', ('CAT17', ['2123712', '2022-04-18', '1', 'CAT17', '2.0', '17.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB02411', ('CB02411', ['2123713', '2022-04-18', '1', 'CB02411', '2.0', '28.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1101', ('CB1101', ['2123714', '2022-04-18', '1', 'CB1101', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1142', ('CB1142', ['2123715', '2022-04-18', '1', 'CB1142', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1143', ('CB1143', ['2123716', '2022-04-18', '1', 'CB1143', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1599', ('CB1599', ['2123717', '2022-04-18', '1', 'CB1599', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1699', ('CB1699', ['2123718', '2022-04-18', '1', 'CB1699', '1.0', '17.0'

-------------------------------------------
Time: 2023-05-14 09:56:22
-------------------------------------------
('CAT17', ('CAT17', ['2132856', '2022-04-23', '29', 'CAT17', '13.0', '24.0', '113', '7', '2022', '16', '2022-113', '16', '2022-16']))
('CB02411', ('CB02411', ['2132857', '2022-04-23', '29', 'CB02411', '18.0', '22.0', '113', '7', '2022', '16', '2022-113', '16', '2022-16']))
('CB1101', ('CB1101', ['2132858', '2022-04-23', '29', 'CB1101', '0.0', '-1.0', '113', '7', '2022', '16', '2022-113', '16', '2022-16']))
('CB1142', ('CB1142', ['2132859', '2022-04-23', '29', 'CB1142', '15.0', '26.0', '113', '7', '2022', '16', '2022-113', '16', '2022-16']))
('CB1143', ('CB1143', ['2132860', '2022-04-23', '29', 'CB1143', '3.0', '23.0', '113', '7', '2022', '16', '2022-113', '16', '2022-16']))
('CB1599', ('CB1599', ['2132861', '2022-04-23', '29', 'CB1599', '5.0', '17.0', '113', '7', '2022', '16', '2022-113', '16', '2022-16']))
('CB1699', ('CB1699', ['2132862', '2022-04-23', '29', 'CB1699', '8.

-------------------------------------------
Time: 2023-05-14 09:56:24
-------------------------------------------
('CAT17', ('CAT17', ['2135808', '2022-04-25', '1', 'CAT17', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB02411', ('CB02411', ['2135809', '2022-04-25', '1', 'CB02411', '3.0', '22.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1101', ('CB1101', ['2135810', '2022-04-25', '1', 'CB1101', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1142', ('CB1142', ['2135811', '2022-04-25', '1', 'CB1142', '2.0', '17.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1143', ('CB1143', ['2135812', '2022-04-25', '1', 'CB1143', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1599', ('CB1599', ['2135813', '2022-04-25', '1', 'CB1599', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1699', ('CB1699', ['2135814', '2022-04-25', '1', 'CB1699', '1.0', '20.0'

-------------------------------------------
Time: 2023-05-14 09:56:26
-------------------------------------------
('CAT17', ('CAT17', ['2147904', '2022-05-02', '1', 'CAT17', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB02411', ('CB02411', ['2147905', '2022-05-02', '1', 'CB02411', '49.0', '17.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1101', ('CB1101', ['2147906', '2022-05-02', '1', 'CB1101', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1142', ('CB1142', ['2147907', '2022-05-02', '1', 'CB1142', '5.0', '20.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1143', ('CB1143', ['2147908', '2022-05-02', '1', 'CB1143', '5.0', '23.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1599', ('CB1599', ['2147909', '2022-05-02', '1', 'CB1599', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1699', ('CB1699', ['2147910', '2022-05-02', '1', 'CB1699', '0.0', '-1.0

...

-------------------------------------------
Time: 2023-05-14 09:59:16
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 09:59:18
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 09:59:20
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 09:59:22
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 09:59:24
-------------------------------------------



In [35]:
ssc.stop(stopSparkContext=False, stopGraceFully=False)

23/05/14 10:00:44 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.ja

-------------------------------------------
Time: 2023-05-14 09:59:26
-------------------------------------------



### RLS N=4

In [36]:
N = 4

batch_interval = 2
[sc,ssc,dstream] = getDStream(spark, batch_interval)
initialStateRDD = sc.parallelize([(x[0], set_RLS_Vanilla_state_from_x(x, N)) for x in different_sensors]) 
dataS=dstream.map(ast.literal_eval)


dataS=dataS.flatMap(lambda x: [(x[3],(x[3],x))]) #x[3] contains the key (sensor)
dataS.pprint()
updatedState = dataS.updateStateByKey(updateFunctionRLS, initialRDD=initialStateRDD)

def save_to_file(rdd):
    """
    Update the state on a file to retrieve data after the stream completed
    """
    # coalesce the RDD into a single partition
    merged_rdd = rdd.coalesce(1)
    # save the merged RDD to a text file
    if not merged_rdd.isEmpty():
        merged_rdd.map(lambda x: f"{x[0]},{(x[1][4]/(x[1][3] if x[1][3] != 0 else 1)):.2f},{x[1][4]},{x[1][3]}") \
            .saveAsTextFile("streaming/RLS_Vanilla_N_4")


updatedState.foreachRDD(lambda x : save_to_file(x))
ssc.start()

-------------------------------------------
Time: 2023-05-14 10:00:56
-------------------------------------------
('CB1101', ('CB1101', ['2120258', '2022-04-16', '1', 'CB1101', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1142', ('CB1142', ['2120259', '2022-04-16', '1', 'CB1142', '3.0', '21.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1143', ('CB1143', ['2120260', '2022-04-16', '1', 'CB1143', '1.0', '27.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1599', ('CB1599', ['2120261', '2022-04-16', '1', 'CB1599', '1.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1699', ('CB1699', ['2120262', '2022-04-16', '1', 'CB1699', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB2105', ('CB2105', ['2120263', '2022-04-16', '1', 'CB2105', '7.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CEE016', ('CEE016', ['2120264', '2022-04-16', '1', 'CEE016', '1.0', '19.0'

-------------------------------------------
Time: 2023-05-14 10:00:58
-------------------------------------------
('CAT17', ('CAT17', ['2123712', '2022-04-18', '1', 'CAT17', '2.0', '17.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB02411', ('CB02411', ['2123713', '2022-04-18', '1', 'CB02411', '2.0', '28.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1101', ('CB1101', ['2123714', '2022-04-18', '1', 'CB1101', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1142', ('CB1142', ['2123715', '2022-04-18', '1', 'CB1142', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1143', ('CB1143', ['2123716', '2022-04-18', '1', 'CB1143', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1599', ('CB1599', ['2123717', '2022-04-18', '1', 'CB1599', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1699', ('CB1699', ['2123718', '2022-04-18', '1', 'CB1699', '1.0', '17.0'

-------------------------------------------
Time: 2023-05-14 10:01:00
-------------------------------------------
('CAT17', ('CAT17', ['2127060', '2022-04-19', '91', 'CAT17', '0.0', '-1.0', '109', '3', '2022', '16', '2022-109', '16', '2022-16']))
('CB02411', ('CB02411', ['2127061', '2022-04-19', '91', 'CB02411', '6.0', '22.0', '109', '3', '2022', '16', '2022-109', '16', '2022-16']))
('CB1101', ('CB1101', ['2127062', '2022-04-19', '91', 'CB1101', '0.0', '-1.0', '109', '3', '2022', '16', '2022-109', '16', '2022-16']))
('CB1142', ('CB1142', ['2127063', '2022-04-19', '91', 'CB1142', '7.0', '19.0', '109', '3', '2022', '16', '2022-109', '16', '2022-16']))
('CB1143', ('CB1143', ['2127064', '2022-04-19', '91', 'CB1143', '0.0', '-1.0', '109', '3', '2022', '16', '2022-109', '16', '2022-16']))
('CB1599', ('CB1599', ['2127065', '2022-04-19', '91', 'CB1599', '1.0', '25.0', '109', '3', '2022', '16', '2022-109', '16', '2022-16']))
('CB1699', ('CB1699', ['2127066', '2022-04-19', '91', 'CB1699', '1.0',

-------------------------------------------
Time: 2023-05-14 10:01:02
-------------------------------------------
('CAT17', ('CAT17', ['2135808', '2022-04-25', '1', 'CAT17', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB02411', ('CB02411', ['2135809', '2022-04-25', '1', 'CB02411', '3.0', '22.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1101', ('CB1101', ['2135810', '2022-04-25', '1', 'CB1101', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1142', ('CB1142', ['2135811', '2022-04-25', '1', 'CB1142', '2.0', '17.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1143', ('CB1143', ['2135812', '2022-04-25', '1', 'CB1143', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1599', ('CB1599', ['2135813', '2022-04-25', '1', 'CB1599', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1699', ('CB1699', ['2135814', '2022-04-25', '1', 'CB1699', '1.0', '20.0'

-------------------------------------------
Time: 2023-05-14 10:01:04
-------------------------------------------
('CAT17', ('CAT17', ['2147904', '2022-05-02', '1', 'CAT17', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB02411', ('CB02411', ['2147905', '2022-05-02', '1', 'CB02411', '49.0', '17.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1101', ('CB1101', ['2147906', '2022-05-02', '1', 'CB1101', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1142', ('CB1142', ['2147907', '2022-05-02', '1', 'CB1142', '5.0', '20.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1143', ('CB1143', ['2147908', '2022-05-02', '1', 'CB1143', '5.0', '23.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1599', ('CB1599', ['2147909', '2022-05-02', '1', 'CB1599', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1699', ('CB1699', ['2147910', '2022-05-02', '1', 'CB1699', '0.0', '-1.0

...

-------------------------------------------
Time: 2023-05-14 10:03:20
-------------------------------------------
('CAT17', ('CAT17', ['2716416', '2023-03-27', '1', 'CAT17', '0.0', '-1.0', '86', '2', '2023', '13', '2023-86', '13', '2023-13']))
('CB02411', ('CB02411', ['2716417', '2023-03-27', '1', 'CB02411', '1.0', '19.0', '86', '2', '2023', '13', '2023-86', '13', '2023-13']))
('CB1101', ('CB1101', ['2716418', '2023-03-27', '1', 'CB1101', '3.0', '13.0', '86', '2', '2023', '13', '2023-86', '13', '2023-13']))
('CB1142', ('CB1142', ['2716419', '2023-03-27', '1', 'CB1142', '1.0', '22.0', '86', '2', '2023', '13', '2023-86', '13', '2023-13']))
('CB1143', ('CB1143', ['2716420', '2023-03-27', '1', 'CB1143', '2.0', '17.0', '86', '2', '2023', '13', '2023-86', '13', '2023-13']))
('CB1599', ('CB1599', ['2716421', '2023-03-27', '1', 'CB1599', '0.0', '-1.0', '86', '2', '2023', '13', '2023-86', '13', '2023-13']))
('CB1699', ('CB1699', ['2716422', '2023-03-27', '1', 'CB1699', '0.0', '-1.0', '86', '2',

-------------------------------------------
Time: 2023-05-14 10:03:22
-------------------------------------------
('CAT17', ('CAT17', ['2569536', '2023-01-01', '1', 'CAT17', '1.0', '9.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB02411', ('CB02411', ['2569537', '2023-01-01', '1', 'CB02411', '8.0', '17.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1101', ('CB1101', ['2569538', '2023-01-01', '1', 'CB1101', '17.0', '15.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1142', ('CB1142', ['2569539', '2023-01-01', '1', 'CB1142', '12.0', '18.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1143', ('CB1143', ['2569540', '2023-01-01', '1', 'CB1143', '16.0', '18.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1599', ('CB1599', ['2569541', '2023-01-01', '1', 'CB1599', '3.0', '19.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1699', ('CB1699', ['2569542', '2023-01-01', '1', 'CB1699', '2.0', '16.0', '1', '1', '2023', '5

-------------------------------------------
Time: 2023-05-14 10:03:24
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 10:03:26
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 10:03:28
-------------------------------------------



In [37]:
ssc.stop(stopSparkContext=False, stopGraceFully=False)

23/05/14 10:04:44 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.ja

-------------------------------------------
Time: 2023-05-14 10:03:30
-------------------------------------------



## RLS Time Information

In [11]:
def RLSstepWithTimeInformation(y, x, n, time_stamp, day, week, year, beta, V, nu):
    """
    Perform one step of the Time augmented RLS 
    """
    x = x.copy()
    x.shape = (1, n)
    x1 = np.append(1, x)
    x1 = np.append(time_stamp, x1)
    x1 = np.append(day, x1)
    x1 = np.append(week, x1)
    x1 = np.append(year, x1)
    x1.shape = (1, n+5)
    V_up = V.dot(x1.T).dot(x1).dot(V)
    V_down = (1.0+float(x1.dot(V).dot(x1.T)))
    
    V = (1.0/nu) * (V - V_up/V_down)
    alpha = V.dot(x1.T)
    yhat = x1.dot(beta)
    err = y - yhat[0]
    
    beta = beta + alpha * err
    return (beta, V, err, yhat[0])

In [12]:
def set_RLS_state_from_xWithTimeInformation(x, previous_value_size):
    """
    Set the initial Time related RLS state from x using N=previous_value_size
    """
    previous_values = np.zeros(previous_value_size)
    nb_predictions = 0
    SSE = 0
    last_prediction = 0
    nb_same_SSE = 0
    v0 = 1
    RLS_parameters = {
        "beta" : np.zeros(previous_value_size + 5), # 5: timestamp, day, week, year, independant term 
        "V" :  np.diag(np.zeros(previous_value_size + 5) + v0),
        "nu" : 1
    }    
    return (x[0],last_prediction, previous_values,nb_predictions,SSE, nb_same_SSE, RLS_parameters)


def updateFunctionRLSWithTimeInformation(newValues, state):
    """   
    Update the state of the corresponding sensor with newValues. 
    Perform the prediction of Time related RLS and updtate the metrics
    @param newValues : the new values from the stream : one row of the corresponding sensor
    @param state : the current state to be updated
    """
    n = 4
    nb_values = len(newValues)
    if nb_values > 0:
        for line in newValues:            
            key, last_prediction, previous_values, nb_predictions, SSE, nb_same_SEE, RLS_parameters = state
            time_stamp = float(line[1][2])
            day_of_week = float(line[1][7]) ### retrieve day of week
            week = float(line[1][6]) ### retrieve day of week
            year = float(line[1][8])
            true_value = float(line[1][4])
            RLS_parameters["beta"].shape=(n+5, 1)
            RLS_parameters["beta"], RLS_parameters["V"], err, prediction = RLSstepWithTimeInformation(true_value, previous_values, n, time_stamp, day_of_week, week, year, RLS_parameters["beta"],RLS_parameters["V"],RLS_parameters["nu"]) # to change for each model
            SSE += math.pow(err, 2)
            nb_predictions +=1
            previous_values = np.concatenate([previous_values[1:],[float(true_value)]])
            date = line[1][1]
            state = (key, prediction, previous_values, nb_predictions, SSE,0, RLS_parameters)
    else:
        key, last_prediction, previous_values, nb_predictions, SSE, nb_same_SEE, RLS_parameters = state 
        nb_same_SEE +=1
        if nb_same_SEE >= 5:
            return None
        state = (key, last_prediction, previous_values, nb_predictions, SSE, nb_same_SEE, RLS_parameters)
    return state

In [13]:
batch_interval = 2
[sc,ssc,dstream] = getDStream(spark, batch_interval)
initialStateRDD = sc.parallelize([(x[0], set_RLS_state_from_xWithTimeInformation(x, 4)) for x in different_sensors]) 
dataS = dstream.map(ast.literal_eval)


dataS = dataS.flatMap(lambda x: [(x[3], (x[3], x))]) #x[3] contains the key (sensor)
dataS.pprint()
updatedState = dataS.updateStateByKey(updateFunctionRLSWithTimeInformation, initialRDD=initialStateRDD)
#updatedState.pprint()

def save_to_file(rdd):
    """
    Update the state on a file to retrieve data after the stream completed
    """
    # coalesce the RDD into a single partition
    merged_rdd = rdd.coalesce(1)
    # save the merged RDD to a text file
    if not merged_rdd.isEmpty():
        merged_rdd.map(lambda x: f"{x[0]},{(x[1][4]/(x[1][3] if x[1][3] != 0 else 1)):.2f},{x[1][4]},{x[1][3]}") \
            .saveAsTextFile("streaming/RLS_TIME_4")

updatedState.foreachRDD(lambda x : save_to_file(x))

In [14]:
ssc.start()

-------------------------------------------
Time: 2023-05-14 11:31:02
-------------------------------------------
('CB1101', ('CB1101', ['2184194', '2022-05-23', '1', 'CB1101', '0.0', '-1.0', '143', '2', '2022', '21', '2022-143', '21', '2022-21']))
('CB1142', ('CB1142', ['2184195', '2022-05-23', '1', 'CB1142', '1.0', '29.0', '143', '2', '2022', '21', '2022-143', '21', '2022-21']))
('CB1143', ('CB1143', ['2184196', '2022-05-23', '1', 'CB1143', '3.0', '23.0', '143', '2', '2022', '21', '2022-143', '21', '2022-21']))
('CB1599', ('CB1599', ['2184197', '2022-05-23', '1', 'CB1599', '0.0', '-1.0', '143', '2', '2022', '21', '2022-143', '21', '2022-21']))
('CB1699', ('CB1699', ['2184198', '2022-05-23', '1', 'CB1699', '1.0', '22.0', '143', '2', '2022', '21', '2022-143', '21', '2022-21']))
('CB2105', ('CB2105', ['2184199', '2022-05-23', '1', 'CB2105', '2.0', '23.0', '143', '2', '2022', '21', '2022-143', '21', '2022-21']))
('CEE016', ('CEE016', ['2184200', '2022-05-23', '1', 'CEE016', '1.0', '18.0'

-------------------------------------------
Time: 2023-05-14 11:31:04
-------------------------------------------
('CAT17', ('CAT17', ['2196288', '2022-05-30', '1', 'CAT17', '0.0', '-1.0', '150', '2', '2022', '22', '2022-150', '22', '2022-22']))
('CB02411', ('CB02411', ['2196289', '2022-05-30', '1', 'CB02411', '3.0', '20.0', '150', '2', '2022', '22', '2022-150', '22', '2022-22']))
('CB1101', ('CB1101', ['2196290', '2022-05-30', '1', 'CB1101', '0.0', '-1.0', '150', '2', '2022', '22', '2022-150', '22', '2022-22']))
('CB1142', ('CB1142', ['2196291', '2022-05-30', '1', 'CB1142', '3.0', '20.0', '150', '2', '2022', '22', '2022-150', '22', '2022-22']))
('CB1143', ('CB1143', ['2196292', '2022-05-30', '1', 'CB1143', '1.0', '25.0', '150', '2', '2022', '22', '2022-150', '22', '2022-22']))
('CB1599', ('CB1599', ['2196293', '2022-05-30', '1', 'CB1599', '1.0', '20.0', '150', '2', '2022', '22', '2022-150', '22', '2022-22']))
('CB1699', ('CB1699', ['2196294', '2022-05-30', '1', 'CB1699', '0.0', '-1.0'

-------------------------------------------
Time: 2023-05-14 11:31:06
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 11:31:08
-------------------------------------------
('CAT17', ('CAT17', ['2208384', '2022-06-06', '1', 'CAT17', '0.0', '-1.0', '157', '2', '2022', '23', '2022-157', '23', '2022-23']))
('CB02411', ('CB02411', ['2208385', '2022-06-06', '1', 'CB02411', '4.0', '20.0', '157', '2', '2022', '23', '2022-157', '23', '2022-23']))
('CB1101', ('CB1101', ['2208386', '2022-06-06', '1', 'CB1101', '0.0', '-1.0', '157', '2', '2022', '23', '2022-157', '23', '2022-23']))
('CB1142', ('CB1142', ['2208387', '2022-06-06', '1', 'CB1142', '2.0', '17.0', '157', '2', '2022', '23', '2022-157', '23', '2022-23']))
('CB1143', ('CB1143', ['2208388', '2022-06-06', '1', 'CB1143', '2.0', '21.0', '157', '2', '2022', '23', '2022-157', '23', '2022-23']))
('CB1599', ('CB1599', ['2208389', '2022-06-06', '1', 'CB1599', '1.0', '25.0', '157', '2', '2022', '23', '2022-157', '23', '2022-23']))
('CB1699', ('CB1699', ['2208390', '2022-06-06', '1', 'CB1699', '1.0', '20.0'

-------------------------------------------
Time: 2023-05-14 11:31:10
-------------------------------------------
('CAT17', ('CAT17', ['2220480', '2022-06-13', '1', 'CAT17', '2.0', '18.0', '164', '2', '2022', '24', '2022-164', '24', '2022-24']))
('CB02411', ('CB02411', ['2220481', '2022-06-13', '1', 'CB02411', '2.0', '26.0', '164', '2', '2022', '24', '2022-164', '24', '2022-24']))
('CB1101', ('CB1101', ['2220482', '2022-06-13', '1', 'CB1101', '0.0', '-1.0', '164', '2', '2022', '24', '2022-164', '24', '2022-24']))
('CB1142', ('CB1142', ['2220483', '2022-06-13', '1', 'CB1142', '0.0', '-1.0', '164', '2', '2022', '24', '2022-164', '24', '2022-24']))
('CB1143', ('CB1143', ['2220484', '2022-06-13', '1', 'CB1143', '1.0', '21.0', '164', '2', '2022', '24', '2022-164', '24', '2022-24']))
('CB1599', ('CB1599', ['2220485', '2022-06-13', '1', 'CB1599', '0.0', '-1.0', '164', '2', '2022', '24', '2022-164', '24', '2022-24']))
('CB1699', ('CB1699', ['2220486', '2022-06-13', '1', 'CB1699', '1.0', '21.0'

...

-------------------------------------------
Time: 2023-05-14 11:33:08
-------------------------------------------
('CB2105', ('CB2105', ['2721337', '2023-03-29', '82', 'CB2105', '26.0', '21.0', '88', '4', '2023', '13', '2023-88', '13', '2023-13']))
('CEE016', ('CEE016', ['2721338', '2023-03-29', '82', 'CEE016', '13.0', '19.0', '88', '4', '2023', '13', '2023-88', '13', '2023-13']))
('CEK049', ('CEK049', ['2721339', '2023-03-29', '82', 'CEK049', '21.0', '20.0', '88', '4', '2023', '13', '2023-88', '13', '2023-13']))
('CEK18', ('CEK18', ['2721340', '2023-03-29', '82', 'CEK18', '10.0', '16.0', '88', '4', '2023', '13', '2023-88', '13', '2023-13']))
('CEK31', ('CEK31', ['2721341', '2023-03-29', '82', 'CEK31', '8.0', '18.0', '88', '4', '2023', '13', '2023-88', '13', '2023-13']))
('CEV011', ('CEV011', ['2721342', '2023-03-29', '82', 'CEV011', '0.0', '-1.0', '88', '4', '2023', '13', '2023-88', '13', '2023-13']))
('CJE181', ('CJE181', ['2721343', '2023-03-29', '82', 'CJE181', '3.0', '17.0', '88',

-------------------------------------------
Time: 2023-05-14 11:33:10
-------------------------------------------
('CAT17', ('CAT17', ['2569536', '2023-01-01', '1', 'CAT17', '1.0', '9.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB02411', ('CB02411', ['2569537', '2023-01-01', '1', 'CB02411', '8.0', '17.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1101', ('CB1101', ['2569538', '2023-01-01', '1', 'CB1101', '17.0', '15.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1142', ('CB1142', ['2569539', '2023-01-01', '1', 'CB1142', '12.0', '18.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1143', ('CB1143', ['2569540', '2023-01-01', '1', 'CB1143', '16.0', '18.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1599', ('CB1599', ['2569541', '2023-01-01', '1', 'CB1599', '3.0', '19.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1699', ('CB1699', ['2569542', '2023-01-01', '1', 'CB1699', '2.0', '16.0', '1', '1', '2023', '5

-------------------------------------------
Time: 2023-05-14 11:33:12
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 11:33:14
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 11:33:16
-------------------------------------------



In [15]:
ssc.stop(stopSparkContext=False, stopGraceFully=False)

23/05/14 11:34:23 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.ja

-------------------------------------------
Time: 2023-05-14 11:33:18
-------------------------------------------

